In [1]:
from __future__ import print_function
import sqlite3
import numpy as np
import model.model
import model.util

In [2]:
ALL_NUM_COLORS = [6, 8, 10]
DB_FILE = "/home/petroff/notebooks/ccs-analysis/results.db"

In [3]:
m = model.model.Model()

In [4]:
def to_lab(color):
    """
    Convert hex color code (without `#`) to rounded LAB.
    """
    rgb = [(int(i[:2], 16), int(i[2:4], 16), int(i[4:], 16)) for i in color]
    lab = [[int(5 * round(float(i)/5)) for i in model.util.convert.convertRGBToLab(c)] for c in rgb]
    return np.array(lab)

In [5]:
# Load survey data
data_lab = {}
targets = {}
min_count = 1e10

conn = sqlite3.connect(DB_FILE)
c = conn.cursor()

for num_colors in ALL_NUM_COLORS:
    count = 0
    data_lab[num_colors] = []
    targets[num_colors] = []
    for row in c.execute(
        "SELECT c1, c2, sp FROM picks WHERE length(c1) = {}".format(num_colors * 7 - 1)
    ):
        count += 1
        # Convert to Jab [CAM02-UCS based]
        lab1 = to_lab(row[0].split(","))
        lab2 = to_lab(row[1].split(","))
        # Add to data arrays
        data_lab[num_colors].append(np.array((lab1, lab2)))
        targets[num_colors].append(row[2] - 1)
    data_lab[num_colors] = np.array(data_lab[num_colors])
    targets[num_colors] = np.array(targets[num_colors])
    min_count = min(min_count, count)
    print(num_colors, count)

conn.close()

6 10347
8 10371
10 1705


In [13]:
# Calculate pair preference scores and check accuracy
score_comp_min_total = []
score_comp_mean_total = []
for nc in ALL_NUM_COLORS:
    score_comp_min = []
    score_comp_mean = []
    for i in range(data_lab[nc].shape[0]):
        score1 = m.scorePalette(data_lab[nc][i, 0])
        score2 = m.scorePalette(data_lab[nc][i, 1])
        pp1 = score1['minScores']['pp']
        pp2 = score2['minScores']['pp']
        score_comp_min.append(pp1 > pp2)
        pp1 = np.mean(np.array(score1['scores'])[:, 2])
        pp1 = np.mean(np.array(score2['scores'])[:, 2])
        score_comp_mean.append(pp1 > pp2)
    print("min", nc, "{:.3f}".format(np.mean(np.array(score_comp_min) ^ targets[nc])))
    print("mean", nc, "{:.3f}".format(np.mean(np.array(score_comp_mean) ^ targets[nc])))
    score_comp_min_total += score_comp_min
    score_comp_mean_total += score_comp_mean
total_targets = np.concatenate((targets[6], targets[8], targets[10]))
print("min total {:.3f}".format(np.mean(np.array(score_comp_min_total) ^ total_targets)))
print("mean total {:.3f}".format(np.mean(np.array(score_comp_mean_total) ^ total_targets)))

min 6 0.498
mean 6 0.524
min 8 0.492
mean 8 0.527
min 10 0.501
mean 10 0.544
min total 0.496
mean total 0.527
